In [58]:
# setup
import pandas as pd
from pathlib import Path
import numpy as np

In [60]:
## path setup
# choose base for uploading
base_folder = Path().resolve().parent

data_folder = base_folder / "data"
cleaned_folder = data_folder / "clean_data"

# create folder
cleaned_folder.mkdir(parents = True, exist_ok = True)

In [62]:
## data cleaning
# create function to get tidy data
def clean_file(path, year, organic, n_states=None, state_names=None, skiprows=0):
    
    # read raw data with no header
    raw = pd.read_excel(path, header=None, skiprows=skiprows, dtype=str) 

    # get number of state columns
    if n_states is None:
        n_states = raw.shape[1] - 1

    # creates new list of column names
    col_names = ["field"] + [f"col_{i}" for i in range(1, n_states + 1)]

    # make sure columns match up
    raw = raw.iloc[:, :len(col_names)]

    # rename columns to standardize dataset
    raw.columns = col_names

    # standardize datatype
    raw["field"] = raw["field"].astype(str).str.strip()

    # drop empty rows
    raw = raw.dropna(how = "all")

    # get category rows
    is_category = raw["field"].str.endswith(":")

    # create category column
    raw["category"] = np.where(
        is_category,
        raw["field"].str.replace(":", "", regex=False).str.strip(),
        np.nan)

    # create item column
    raw["item"] = np.where(~is_category, raw["field"], np.nan)

    # fill categories downwards
    raw["category"] = raw["category"].ffill()

    # keep rows that have data
    items = raw[raw["item"].notna()].copy()

    # get data to long format
    long_df = items.melt(
        id_vars = ["category", "item"],
        value_vars = [f"col_{i}" for i in range(1, n_states + 1)],
        var_name = "state_index",
        value_name = "value")

    # get values and convert to integers
    long_df["state_index"] = long_df["state_index"].str.extract(r"col_(\d+)", expand=False).astype(int)

    # if/else to create state names
    if state_names is not None:
        long_df["State"] = long_df["state_index"].apply(lambda i: state_names[i-1])
    else:
        long_df["State"] = long_df["state_index"].apply(lambda i: f"State_{i}")

    # convert to numeric datatype
    long_df["value"] = pd.to_numeric(long_df["value"], errors="coerce")

    # adds year column
    long_df["year"] = year

    # adds organic indicator
    long_df["organic"] = organic

    # return the function
    return long_df[["category", "item", "State", "value", "year", "organic"]]

In [64]:
## data cleaning
# create function to build dataset
def build_dataset(path, year, organic, states, skiprows=0):
    
    # call clean_file() function to get tidy data
    long_df = clean_file(path = path, year = year, organic = organic, n_states = len(states),
                         state_names = states, skiprows = skiprows)

    # select which variables to keep and corretly name them
    select_variables = {"Milk sold": "milk_sold",
                        "Total, gross value of production": "gross_value",
                        "Total, feed costs": "feed_costs",
                        "Veterinary and medicine": "vet_costs",
                        "Bedding and litter": "bedding_litter_costs",
                        "Marketing": "marketing_costs",
                        "Custom services": "service_costs",
                        "Fuel, lube, and electricity": "utility_costs",
                        "Repairs": "repair_costs",
                        "Total, operating cost": "total_operating_costs",
                        "Total, allocated overhead": "overhead",
                        "Total costs listed": "total_costs",
                        "Milk cows (head per farm)": "cows_per_farm",
                        "Output per cow (pounds)": "output_per_cow"}

    variables = list(select_variables.keys())

    # filter long_df with select variables
    sub = long_df[long_df["item"].isin(variables)].copy()

    # convert organic to yes/no
    sub["Organic"] = np.where(sub["organic"] == 1, "Yes", "No")

    # pivot from long data to wide data table so there's only 1 row for data
    wide = (sub.pivot_table(index = ["year", "State", "Organic"],
                            columns = "item", values = "value",
                            aggfunc = "first").reset_index())

    # rename columns
    wide = wide.rename(columns = select_variables)
    wide = wide.rename(columns = {"year": "Year"})

    # reorder columns of data
    ordered_cols = ["Year", "State", "Organic"] + list(select_variables.values())

    # return new and clean dataset
    final = wide[ordered_cols]
    return final

In [66]:
## load data

# selected states
states = ["Minnesota", "New York",
          "Pennsylvania", "Vermont", 
          "Wisconsin"]

# 2005 organic milk data
org_2005_table = build_dataset(path = data_folder / "milkorg2005.xlsx", 
                               year = 2005, organic = 1,
                               states = states, skiprows = 0)

# 2010 organic milk data
org_2010_table = build_dataset(path = data_folder / "milkorg11.xlsx",
                               year = 2010, organic = 1,
                               states = states, skiprows = 0)

# 2016 organic milk data
org_2016_table = build_dataset(path = data_folder / "milkorg2016-2.xlsx",
                               year = 2016, organic = 1,
                               states = states, skiprows = 0)

# 2005 conventional milk data
conv_2005_table = build_dataset(path = data_folder / "milkconv2005.xlsx",
                                year = 2005, organic = 0,
                                states = states, skiprows = 0)

# 2010 conventional milk data
conv_2010_table = build_dataset(path = data_folder / "milkconv11.xlsx",
                                year = 2010, organic = 0,
                                states = states, skiprows = 0)

# 2016 conventional milk data
conv_2016_table = build_dataset(path = data_folder / "milkconv2016-2.xlsx",
                                year = 2016, organic = 0,
                                states = states, skiprows = 0)

In [68]:
## merge datasets

# organic data
org_milk = pd.concat([org_2005_table, org_2010_table, org_2016_table], ignore_index = True)

# conventional data
conv_milk = pd.concat([conv_2005_table, conv_2010_table, conv_2016_table], ignore_index = True)

# all data
milk_df = pd.concat([org_milk, conv_milk], ignore_index=True)

# convert organic variable 
milk_df["Organic"] = milk_df["Organic"].map({"Yes": "1", "No": "0"})

# look at dataset
milk_df

item,Year,State,Organic,milk_sold,gross_value,feed_costs,vet_costs,bedding_litter_costs,marketing_costs,service_costs,utility_costs,repair_costs,total_operating_costs,overhead,total_costs,cows_per_farm,output_per_cow
0,2005,Minnesota,1,20.88,23.17,10.11,0.54,0.38,0.43,0.49,0.89,1.07,14.30,16.31,30.61,59.0,13058.0
1,2005,New York,1,24.03,26.87,13.08,0.35,0.37,0.24,0.10,1.12,0.94,16.63,21.37,38.00,53.0,10956.0
2,2005,Pennsylvania,1,24.23,26.85,13.23,0.68,0.46,0.31,0.23,0.73,0.94,17.10,14.61,31.71,46.0,12177.0
3,2005,Vermont,1,24.23,26.87,13.61,0.81,0.52,0.24,0.22,0.98,0.87,17.62,18.88,36.50,62.0,11982.0
4,2005,Wisconsin,1,21.19,23.63,13.47,0.60,0.39,0.22,0.33,0.84,0.85,17.11,14.95,32.07,65.0,13101.0
5,2010,Minnesota,1,25.17,27.81,14.01,0.49,0.47,0.25,0.60,1.20,0.65,17.89,21.99,39.88,36.0,10542.0
6,2010,New York,1,24.29,27.22,11.58,0.62,0.52,0.39,0.68,1.24,1.19,16.39,18.10,34.49,73.0,12880.0
7,2010,Pennsylvania,1,28.37,30.92,14.05,0.51,0.44,0.23,0.41,1.44,1.55,18.81,20.52,39.33,72.0,12247.0
8,2010,Vermont,1,25.97,28.61,15.88,0.57,0.50,0.23,0.45,1.32,1.96,21.05,22.45,43.50,60.0,12145.0
9,2010,Wisconsin,1,27.90,30.89,14.75,0.80,0.58,0.19,0.93,1.09,1.30,19.85,21.15,41.00,56.0,12940.0


In [70]:
# export file
output_path = cleaned_folder / "milk_df_clean.xlsx"

milk_df.to_excel(output_path, index = False)